### Beef Grading System V-0

 This is the prototype of the Beef Grading System to be used as an MVP for my submission to Create-X. It's form will be that of a CNN, trained on beef data from the store,(Costco, most likely)

In [2]:
import torch
print(torch.__version__)
print("CUDA Available:", torch.cuda.is_available())


2.6.0+cpu
CUDA Available: False


In [3]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib as plt
import numpy as np
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader

# Define paths to your dataset
data_dir = 'C:/Users/andyd/OneDrive/Desktop/Beef-V.0/BeefImages'
classes = ["Prime", "Choice", "Select"]

# Image preprocessing and augmentation
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomRotation(10),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load dataset
train_dataset = datasets.ImageFolder(root=data_dir, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Load pre-trained ResNet model
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(classes))

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
def train_model(model, train_loader, criterion, optimizer, epochs=10):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader):.4f}")

print("Dataset and model are ready for training!")





c:\Users\andyd\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\andyd\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\andyd/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:05<00:00, 8.43MB/s]


Dataset and model are ready for training!


In [4]:
train_model(model, train_loader, criterion, optimizer, epochs=10)

Epoch 1/10, Loss: 0.6710
Epoch 2/10, Loss: 0.3185
Epoch 3/10, Loss: 0.0480
Epoch 4/10, Loss: 0.1743
Epoch 5/10, Loss: 0.3372
Epoch 6/10, Loss: 0.0007
Epoch 7/10, Loss: 0.0239
Epoch 8/10, Loss: 0.0686
Epoch 9/10, Loss: 0.0145
Epoch 10/10, Loss: 0.1225


In [6]:
test_data_dir = "C:/Users/andyd/OneDrive/Desktop/Beef-V.0/TestImages"
test_dataset = datasets.ImageFolder(root=test_data_dir, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
    print(f"Test Accuracy: {100 * correct / total:.2f}%")

evaluate_model(model, test_loader)


Test Accuracy: 57.14%
